In [90]:
library(GenomicRanges)
library(dplyr)
library(data.table)
library(ggplot2)
library(viridis)
library(readxl)
library(pheatmap)
library(RColorBrewer)
library(tidyr)
library(gridExtra)
library(corrplot)
library(tibble)
library(ggrepel)
library(ggpubr)
# library(DESeq2)
library(DEFormats)
library(BiocParallel)
register(MulticoreParam(4))
library(chromVAR)
## library(motifmatchr) ##not used by far
library(GenomicFeatures)
library(BSgenome.Hsapiens.UCSC.hg38)
library(ChIPseeker)
library(TxDb.Hsapiens.UCSC.hg38.knownGene)
library(org.Hs.eg.db)
library(pheatmap)
library(RColorBrewer)
library(tidyr)
library(gridExtra)
library(ggrepel)
library(limma)
library(ggdendro)
library(ggfortify)
library(ggpubr)
library(magrittr)
library(Seurat)

In [ ]:
## Read in Clinical data
propscore = fread("CART_CUTRUN_Project/meta/stratify_cd8_tcm.csv", sep = ",", header = TRUE) %>% data.frame()
#fread("CART_CUTRUN_Project/meta/TCM_CD8_PFS_propscore_propmatch_update20230330.csv", sep = ",", header = TRUE) %>% data.frame()
propscore %>% arrange(xnumber)

In [ ]:
# metadata_path = "CART_CUTRUN_Project/meta/"
# clinical_data = fread(paste0(metadata_path, "NHL_DL2_FluCy_Propscore_linear_model_CandR_only_pfs.csv"), sep = ",", header = TRUE) %>% data.frame
# head(clinical_data)

# clinical_data %>% colnames
# clinical_select = c("xnumber", "cd8_max", "cd4_max", "log_cd8_max", "propscore_cd8_max", "propscore", "median_cd8_max", "preld_ldh", "pfscode",	"pfsdaysafterfirstcartcell", "pfs_late_code", "log_pfs_days")
# clinical_data_select = clinical_data[, clinical_select]
# clinical_data_select
patientRaw = read_xlsx("CART_CUTRUN_Project/report/CART_CUTRUN/analysis/20201028_NHL_CUTandRUN_PatientData.xlsx")
patientRaw %>% dim
patientRaw

In [ ]:
## Read in CUTRUN data. All the 30 patients data are included
patientRaw = read_xlsx("CART_CUTRUN_Project/report/CART_CUTRUN/analysis/20201028_NHL_CUTandRUN_PatientData.xlsx")
cm_ind = which(patientRaw$Xnumber %in% propscore$xnumber) #which(patientRaw$`CD8+ T cell isolation` == "CD8TCM") #1:nrow(patientRaw) #
patientRaw$Xnumber[cm_ind]
patient_id_list = patientRaw$Xnumber[cm_ind] %>% tolower
patient_id_list

## patient data
outPath = "CART_CUTRUN_Project/results/CUTANDRUN/analysis/"
histList = c("H3K27me3", "H3K4me2")

countMat = readRDS(file = paste0(outPath, "/RData/TSS_neighbors_chromVar_count_NHLpatient_noChrXYM_countMat.rds"))
normMat = readRDS(file = paste0(outPath, "/RData/TSS_neighbors_chromVar_count_NHLpatient_noChrXYM_normMat.rds")) ## RC
designInfo = readRDS(file = paste0(outPath, "/RData/TSS_neighbors_chromVar_count_NHLpatient_noChrXYM_designInfo.rds"))

cr_countMat = countMat
cr_countMat$H3K27me3 = countMat$H3K27me3[, cm_ind]
cr_countMat$H3K4me2 = countMat$H3K4me2[, cm_ind]
colnames(cr_countMat$H3K27me3) = paste0(colnames(cr_countMat$H3K27me3), "_PT")
colnames(cr_countMat$H3K4me2) = paste0(colnames(cr_countMat$H3K4me2), "_PT")

cr_normMat = normMat
cr_normMat$H3K27me3 = log2(normMat$H3K27me3[, patientRaw$Xnumber[cm_ind]]+1)
cr_normMat$H3K4me2 = log2(normMat$H3K4me2[, patientRaw$Xnumber[cm_ind]]+1)
colnames(cr_normMat$H3K27me3) = paste0(colnames(cr_normMat$H3K27me3), "_PT")
colnames(cr_normMat$H3K4me2) = paste0(colnames(cr_normMat$H3K4me2), "_PT")


pt_designInfo = designInfo %>% dplyr::filter(exps %in% patientRaw$Xnumber[cm_ind])
head(cr_normMat$H3K27me3)
pt_designInfo %>% arrange(hist, exps)



In [ ]:
## Read in RNA data.
## Patient data infusion product of CM-derived cells 
inPath = "CART_CUTRUN_Project/results/RNAseq/process/NHL_RSEM_gencode_v21/"

rna_countMat = c()
pt_name_tmp_list = c()
for(sample in paste0("Sample_", patient_id_list, "_IP_CD8-CAR")){
  if(file.exists(paste0(inPath, sample, "/RSEM.genes.results"))){
    print(sample)
    tmp = fread(paste0(inPath, sample, "/RSEM.genes.results")) %>% dplyr::select(expected_count)
    rna_countMat = cbind(rna_countMat, round(tmp))
    pt_name_tmp_list = c(pt_name_tmp_list, sample)
  }
}
rna_countMat = data.frame(rna_countMat)
colnames(rna_countMat) = paste0(pt_name_tmp_list, "_PT") #paste0(patient_id_list, "_PT")

geneID = fread(paste0(inPath, sample, "/RSEM.genes.results"))$gene_id
rna_countMat$gene_name = geneID 

head(rna_countMat)
dim(rna_countMat)


rownames(rna_countMat) = rna_countMat$gene_name
rna_countMat = rna_countMat %>% dplyr::select(-gene_name)
head(rna_countMat)
dim(rna_countMat)

rna_normMat = log2(rna_countMat + 1)

In [ ]:
## PCA tcm: 5000-2000-10000-10000
rna_obj_tmp = CreateSeuratObject(counts = rna_countMat, project = "cart", min.cells = 0, min.features = 10) %>%  
NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) 

rna_obj = rna_obj_tmp %>% 
FindVariableFeatures(., selection.method = "vst", nfeatures = 5000) %>% 
ScaleData()
rna_obj = RunPCA(rna_obj, features = VariableFeatures(object = rna_obj), npcs = 6)

k27_obj_tmp = CreateSeuratObject(counts = cr_countMat$H3K27me3, project = "cart", min.cells = 0, min.features = 10) %>%  
NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000)

k27_obj = k27_obj_tmp %>% 
FindVariableFeatures(., selection.method = "vst", nfeatures = 2000) %>%  #
ScaleData()

k27_obj = RunPCA(k27_obj, features = VariableFeatures(object = k27_obj), npcs = 6)

k4_obj_tmp = CreateSeuratObject(counts = cr_countMat$H3K4me2, project = "cart", min.cells = 0, min.features = 10) %>%  
NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) 
k4_obj = k4_obj_tmp %>% 
FindVariableFeatures(., selection.method = "vst", nfeatures = 10000) %>%  #round(nrow(k4_m)/10*8)
ScaleData()
k4_obj = RunPCA(k4_obj, features = VariableFeatures(object = k4_obj), npcs = 6)

cr_obj = CreateSeuratObject(counts = rbind(cr_countMat$H3K27me3, cr_countMat$H3K4me2), project = "cart", min.cells = 0, min.features = 10) %>%  
NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) %>% 
FindVariableFeatures(., selection.method = "vst", nfeatures = 10000) %>%  #round((nrow(k27_m) + nrow(k4_m))/10*8)
ScaleData()
cr_obj = RunPCA(cr_obj, features = VariableFeatures(object = cr_obj), npcs = 6)


# cr_obj = CreateSeuratObject(counts = rbind(k27_obj_tmp@assays$RNA@data, k4_obj_tmp@assays$RNA@data), project = "cart", min.cells = 0, min.features = 10) %>%  
# # NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) %>% 
# FindVariableFeatures(., selection.method = "vst", nfeatures = 1000) %>%  #round((nrow(k27_m) + nrow(k4_m))/10*8)
# ScaleData()
# cr_obj = RunPCA(cr_obj, features = VariableFeatures(object = cr_obj), npcs = 12)


In [ ]:
# rna_proplabel = left_join(
#     data.frame(xnumber = rna_obj %>% colnames %>% gsub("Sample_", "", .) %>% gsub("_IP_CD8.*", "", .) %>% toupper()), 
#     propscore[, c("xnumber", "propmatch_cd8", "propmatch_pfs")]
# )

# rna_proplabel$propmatch_cd8[is.na(rna_proplabel$propmatch_cd8)] = "Missing"
# rna_proplabel$propmatch_pfs[is.na(rna_proplabel$propmatch_pfs)] = "Missing"

# rna_proplabel$propmatch_cd8 = factor(rna_proplabel$propmatch_cd8, levels = c(0, 1, "Missing"), labels = c("Low_Expander", "High_Expander", "Missing"))
# rna_proplabel$propmatch_pfs = factor(rna_proplabel$propmatch_pfs, levels = c(0, 1, "Missing"), labels = c("Early_Progressor", "Late_Progressor", "Missing"))
# rna_proplabel

# k4_proplabel = left_join(
#     data.frame(xnumber = k4_obj %>% colnames %>% gsub("_.*", "", .) %>% toupper()), 
#     propscore[, c("xnumber", "propmatch_cd8", "propmatch_pfs")]
# )

# k4_proplabel$propmatch_cd8[is.na(k4_proplabel$propmatch_cd8)] = "Missing"
# k4_proplabel$propmatch_pfs[is.na(k4_proplabel$propmatch_pfs)] = "Missing"

# k4_proplabel$propmatch_cd8 = factor(k4_proplabel$propmatch_cd8, levels = c(0, 1, "Missing"), labels = c("Low_Expander", "High_Expander", "Missing"))
# k4_proplabel$propmatch_pfs = factor(k4_proplabel$propmatch_pfs, levels = c(0, 1, "Missing"), labels = c("Early_Progressor", "Late_Progressor", "Missing"))
# k4_proplabel

# k27_proplabel = left_join(
#     data.frame(xnumber = k27_obj %>% colnames %>% gsub("_.*", "", .) %>% toupper()), 
#     propscore[, c("xnumber", "propmatch_cd8", "propmatch_pfs")]
# )

# k27_proplabel$propmatch_cd8[is.na(k27_proplabel$propmatch_cd8)] = "Missing"
# k27_proplabel$propmatch_pfs[is.na(k27_proplabel$propmatch_pfs)] = "Missing"

# k27_proplabel$propmatch_cd8 = factor(k27_proplabel$propmatch_cd8, levels = c(0, 1, "Missing"), labels = c("Low_Expander", "High_Expander", "Missing"))
# k27_proplabel$propmatch_pfs = factor(k27_proplabel$propmatch_pfs, levels = c(0, 1, "Missing"), labels = c("Early_Progressor", "Late_Progressor", "Missing"))
# k27_proplabel



In [ ]:
rna_proplabel = left_join(
    data.frame(xnumber = rna_obj %>% colnames %>% gsub("Sample_", "", .) %>% gsub("_IP_CD8.*", "", .) %>% toupper()), 
    propscore[, c("xnumber", "stratify_cd8_all")]
)
rna_proplabel$stratify_cd8_all[is.na(rna_proplabel$stratify_cd8_all)] = "Missing"
rna_proplabel$stratify_cd8_all = factor(rna_proplabel$stratify_cd8_all, levels = c("Low expander", "High expander", "Missing"), labels = c("Low_Expander", "High_Expander", "Missing"))
rna_proplabel

k4_proplabel = left_join(
    data.frame(xnumber = k4_obj %>% colnames %>% gsub("_.*", "", .) %>% toupper()), 
    propscore[, c("xnumber", "stratify_cd8_all")]
)
k4_proplabel$stratify_cd8_all[is.na(k4_proplabel$stratify_cd8_all)] = "Missing"
k4_proplabel$stratify_cd8_all = factor(k4_proplabel$stratify_cd8_all, levels = c("Low expander", "High expander", "Missing"), labels = c("Low_Expander", "High_Expander", "Missing"))
k4_proplabel

k27_proplabel = left_join(
    data.frame(xnumber = k27_obj %>% colnames %>% gsub("_.*", "", .) %>% toupper()), 
    propscore[, c("xnumber", "stratify_cd8_all")]
)
k27_proplabel$stratify_cd8_all[is.na(k27_proplabel$stratify_cd8_all)] = "Missing"
k27_proplabel$stratify_cd8_all = factor(k27_proplabel$stratify_cd8_all, levels = c("Low expander", "High expander", "Missing"), labels = c("Low_Expander", "High_Expander", "Missing"))
k27_proplabel



In [ ]:
p11 = Embeddings(rna_obj, reduction = "pca")[, 1:2] %>% data.frame %>% 
mutate(Expand = rna_proplabel$stratify_cd8_all, Xnumber = rna_proplabel$xnumber) %>% #dplyr::filter(Expand != "Missing") %>%
ggplot(aes(x = PC_1, y = PC_2, color = Expand, label = Xnumber)) +
geom_point(size = 5) +
# ggrepel::geom_label_repel(size =3) +
theme_bw(base_size = 20) +
scale_color_brewer(palette = "Set1") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("RNA-seq") +
theme(legend.position = "top")

p21 = Embeddings(k27_obj, reduction = "pca")[, 1:4] %>% data.frame %>% 
mutate(Expand = k27_proplabel$stratify_cd8_all, Xnumber = k27_proplabel$xnumber) %>% #dplyr::filter(Expand != "Missing") %>%
ggplot(aes(x = PC_1, y = PC_2, color = Expand, label = Xnumber)) +
geom_point(size = 5) +
# ggrepel::geom_label_repel(size =3) +
theme_bw(base_size = 20) +
scale_color_brewer(palette = "Set1") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K27me3") +
theme(legend.position = "top")

p31 = Embeddings(k4_obj, reduction = "pca")[, 1:4] %>% data.frame %>% 
mutate(Expand = k4_proplabel$stratify_cd8_all, Xnumber = k4_proplabel$xnumber) %>% #dplyr::filter(Expand != "Missing") %>%
ggplot(aes(x = PC_1, y = PC_2, color = Expand, label = Xnumber)) +
geom_point(size = 5) +
# ggrepel::geom_label_repel(size = 3) +
theme_bw(base_size = 20) +
scale_color_brewer(palette = "Set1") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K4me2") +
theme(legend.position = "top")

p41 = Embeddings(cr_obj, reduction = "pca")[, 1:4] %>% data.frame %>% 
mutate(Expand = k4_proplabel$stratify_cd8_all, Xnumber = k4_proplabel$xnumber) %>% #dplyr::filter(Expand != "Missing") %>%
ggplot(aes(x = PC_1, y = PC_2, color = Expand, label = Xnumber)) +
geom_point(size = 5) +
# ggrepel::geom_label_repel(size = 3) +
theme_bw(base_size = 20) +
scale_color_brewer(palette = "Set1") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K27me3+H3K4me2") +
theme(legend.position = "top")


options(repr.plot.width=20, repr.plot.height=7)
ggarrange(p11, p21, p31, p41, nrow = 1, ncol = 4, common.legend = TRUE)
ggsave("CART_CUTRUN_Project/results/paper_figure/fig7/stratify_cd8_tcm_rna_k27_k4_cr_low_high_expander.pdf", width = 20, height = 7)

In [ ]:
p11 = Embeddings(rna_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(Expand = rna_proplabel$propmatch_cd8) %>% dplyr::filter(Expand != "Missing") %>%
ggplot(aes(x = PC_1, y = PC_2, color = Expand)) +
geom_point(size = 5) +
theme_bw(base_size = 20) +
scale_color_brewer(palette = "Set1") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("RNA-seq") +
theme(legend.position = "top")


p12 = Embeddings(rna_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(Progress = rna_proplabel$propmatch_pfs) %>% dplyr::filter(Progress != "Missing") %>%
ggplot(aes(x = PC_1, y = PC_2, color = Progress)) +
geom_point(size = 5) +
theme_bw(base_size = 20) +
scale_color_brewer(palette = "Dark2") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("RNA-seq") +
theme(legend.position = "top")


p21 = Embeddings(k27_obj, reduction = "pca")[, 1:4] %>% data.frame %>% mutate(Expand = k27_proplabel$propmatch_cd8) %>% dplyr::filter(Expand != "Missing") %>%
ggplot(aes(x = PC_1, y = PC_2, color = Expand)) +
geom_point(size = 5) +
theme_bw(base_size = 20) +
scale_color_brewer(palette = "Set1") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K27me3") +
theme(legend.position = "top")



p22 = Embeddings(k27_obj, reduction = "pca")[, 1:4] %>% data.frame %>% mutate(Progress = k27_proplabel$propmatch_pfs) %>% dplyr::filter(Progress != "Missing") %>%
ggplot(aes(x = PC_1, y = PC_2, color = Progress)) +
geom_point(size = 5) +
theme_bw(base_size = 20) +
scale_color_brewer(palette = "Dark2") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K27me3") +
theme(legend.position = "top")


p31 = Embeddings(k4_obj, reduction = "pca")[, 1:4] %>% data.frame %>% mutate(Expand = k4_proplabel$propmatch_cd8) %>% dplyr::filter(Expand != "Missing") %>%
ggplot(aes(x = PC_1, y = PC_2, color = Expand)) +
geom_point(size = 5) +
theme_bw(base_size = 20) +
scale_color_brewer(palette = "Set1") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K4me2") +
theme(legend.position = "top")



p32 = Embeddings(k4_obj, reduction = "pca")[, 1:4] %>% data.frame %>% mutate(Progress = k4_proplabel$propmatch_pfs) %>% dplyr::filter(Progress != "Missing") %>%
ggplot(aes(x = PC_1, y = PC_2, color = Progress)) +
geom_point(size = 5) +
theme_bw(base_size = 20) +
scale_color_brewer(palette = "Dark2") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K4me2") +
theme(legend.position = "top")



p41 = Embeddings(cr_obj, reduction = "pca")[, 1:4] %>% data.frame %>% mutate(Expand = k4_proplabel$propmatch_cd8) %>% dplyr::filter(Expand != "Missing") %>%
ggplot(aes(x = PC_1, y = PC_2, color = Expand)) +
geom_point(size = 5) +
theme_bw(base_size = 20) +
scale_color_brewer(palette = "Set1") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K27me3+H3K4me2") +
theme(legend.position = "top")



p42 = Embeddings(cr_obj, reduction = "pca")[, 1:4] %>% data.frame %>% mutate(Progress = k4_proplabel$propmatch_pfs) %>% dplyr::filter(Progress != "Missing") %>%
ggplot(aes(x = PC_1, y = PC_2, color = Progress)) +
geom_point(size = 5) +
theme_bw(base_size = 20) +
scale_color_brewer(palette = "Dark2") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K27me3+H3K4me2") +
theme(legend.position = "top")


options(repr.plot.width=20, repr.plot.height=7)
ggarrange(p11, p21, p31, p41, nrow = 1, ncol = 4, common.legend = TRUE)

ggarrange(p12, p22, p32, p42, nrow = 1, ncol = 4, common.legend = TRUE)


In [30]:
## Differential detection
## propmatch_cd8

responseList = c("Low_Expander", "High_Expander")
hdList <- patient_id_list
histList <- c("H3K27me3", "H3K4me2")
target = k4_proplabel

## Experimental design
treat <- factor(target$propmatch_cd8)
design <- model.matrix(~0 + treat)
colnames(design) <- levels(treat)
contrast <- makeContrasts(High_Low = High_Expander - Low_Expander, levels = design)

In [ ]:
outpath = "CART_CUTRUN_Project/results/CUTANDRUN/analysis/PatientData/"
histList = c("H3K27me3", "H3K4me2")
thres_list = list("H3K27me3" = 5, "H3K4me2" = 5)
results = vector("list", length(histList))
results_fullInfo = vector("list", length(histList))
voomDDS = vector("list", length(histList))
for(hist in histList){
  ## Filter and delete low expressed genes
    selectR = which(rowSums(cr_countMat[[hist]]) > 50) ## remove low count genes
    dataS = cr_countMat[[hist]][selectR, ]
    voomDDS[[hist]] = voom(counts = dataS, design = design, normalize.method = "cyclicloess", plot = FALSE)
    ## option 1 using voomDDS option2 using normDDS as normalized input.
    inputDDS = voomDDS[[hist]]
    ## corfit$consensus
    fit = lmFit(inputDDS, design)
    fitContrast = contrasts.fit(fit, contrast)
    fitBayes = eBayes(fitContrast, robust = TRUE)
    results[[hist]] = list()
    results_fullInfo[[hist]] = list()
  
    ## Results
    res = topTable(fit = fitBayes, adjust.method = 'fdr', coef = 1, number = nrow(inputDDS), sort = 'P') %>% data.table
    # res = data.table(PeakRegion = rownames(res), res)
    res[, Significance := ifelse((adj.P.Val <= 0.1 & sign(logFC) == 1 & abs(logFC) >= 1), 'Up',
                            ifelse((adj.P.Val <= 0.1 & sign(logFC) == -1 & abs(logFC) >= 1), 'Down', 'notDE'))]
    colnames(cr_normMat[[hist]]) = paste0(k4_proplabel$xnumber, "_", k4_proplabel$propmatch_cd8)

    results[[hist]] = left_join(res, cr_normMat[[hist]] %>% data.frame %>% dplyr::mutate(ID = rownames(cr_normMat[[hist]])), by = "ID") %>% data.frame
    results[[hist]]$Average_LowExpander = rowMeans(results[[hist]][, which(stringr::str_detect(colnames(results[[hist]]), "Low_Expander"))])
    results[[hist]]$Average_HighExpander = rowMeans(results[[hist]][, which(stringr::str_detect(colnames(results[[hist]]), "High_Expander"))])
    results[[hist]]$Average_LowExpander_Group = 0
    results[[hist]]$Average_HighExpander_Group = 0
    results[[hist]]$Average_LowExpander_Group[results[[hist]]$Average_LowExpander > thres_list[[hist]]] = 1
    results[[hist]]$Average_HighExpander_Group[results[[hist]]$Average_HighExpander > thres_list[[hist]]] = 1
    
    write.table(results[[hist]], file = paste0(outPath, '/DE_', hist, '_patient_TSSregions_Low_High_Expander_adj0.1_logFC2.csv'), quote = FALSE, row.names = FALSE, sep = ",")
    
}


In [ ]:
## RNA-seq Differential detection
## propmatch_cd8

responseList = c("Low_Expander", "High_Expander")
hdList <- patient_id_list
target = rna_proplabel

## Experimental design
treat <- factor(target$propmatch_cd8)
design <- model.matrix(~0 + treat)
colnames(design) <- levels(treat)
contrast <- makeContrasts(High_Low = High_Expander - Low_Expander, levels = design)

rna_countMat %>% colnames()
rna_proplabel
design
contrast
paste0(rna_proplabel$xnumber, "_", rna_proplabel$propmatch_cd8)

In [84]:
outpath = "CART_CUTRUN_Project/results/CUTANDRUN/analysis/PatientData/"
## Filter and delete low expressed genes
selectR = which(rowSums(rna_countMat) > 50) ## remove low count genes
dataS = rna_countMat[selectR, ]
voomDDS = voom(counts = dataS, design = design, normalize.method = "cyclicloess", plot = FALSE)

## option 1 using voomDDS option2 using normDDS as normalized input.
inputDDS = voomDDS
## corfit$consensus
fit = lmFit(inputDDS, design)
fitContrast = contrasts.fit(fit, contrast)
fitBayes = eBayes(fitContrast, robust = TRUE)

## Results
res = topTable(fit = fitBayes, adjust.method = 'fdr', coef = 1, number = nrow(inputDDS), sort = 'P')
res = data.table(ID = rownames(res), res)
res[, Significance := ifelse((adj.P.Val <= 0.1 & sign(logFC) == 1 & abs(logFC) >= 1), 'Up',
                        ifelse((adj.P.Val <= 0.1 & sign(logFC) == -1 & abs(logFC) >= 1), 'Down', 'notDE'))]
colnames(rna_normMat) = paste0(rna_proplabel$xnumber, "_", rna_proplabel$propmatch_cd8)

rna_results = left_join(res, rna_normMat %>% data.frame %>% dplyr::mutate(ID = rownames(rna_normMat)), by = "ID") %>% data.frame
rna_results$Average_LowExpander = rowMeans(rna_results[, which(stringr::str_detect(colnames(rna_results), "Low_Expander"))])
rna_results$Average_HighExpander = rowMeans(rna_results[, which(stringr::str_detect(colnames(rna_results), "High_Expander"))])
rna_results$Average_LowExpander_Group = 0
rna_results$Average_HighExpander_Group = 0
rna_results$Average_LowExpander_Group[rna_results$Average_LowExpander > 5] = 1
rna_results$Average_HighExpander_Group[rna_results$Average_HighExpander > 5] = 1

write.table(rna_results, file = paste0(outPath, '/DE_RNAseq_patient_TSSregions_Low_High_Expander_adj0.1_logFC2.csv'), quote = FALSE, row.names = FALSE, sep = ",")




In [97]:
## Differential detection for stratify_cd8_tcm
## stratify_cd8_all

responseList = c("Low_Expander", "High_Expander")
hdList <- patient_id_list
histList <- c("H3K27me3", "H3K4me2")
target = k4_proplabel

## Experimental design
treat <- factor(target$stratify_cd8_all)
design <- model.matrix(~0 + treat)
colnames(design) <- levels(treat)
contrast <- makeContrasts(High_Low = High_Expander - Low_Expander, levels = design)

In [99]:
cr_normMat[["H3K27me3"]] = k27_obj@assays$RNA@data %>% data.frame
cr_normMat[["H3K4me2"]] = k4_obj@assays$RNA@data %>% data.frame
rna_normMat = rna_obj@assays$RNA@data %>% data.frame

rownames(rna_normMat) = rownames(rna_normMat) %>% sub("-", "_", ., fixed = TRUE)

In [100]:
outpath = "CART_CUTRUN_Project/results/CUTANDRUN/analysis/PatientData/"
histList = c("H3K27me3", "H3K4me2")
thres_list = list("H3K27me3" = 3, "H3K4me2" = 2.5)
results = vector("list", length(histList))
results_fullInfo = vector("list", length(histList))
voomDDS = vector("list", length(histList))
for(hist in histList){
  ## Filter and delete low expressed genes
    selectR = which(rowSums(cr_countMat[[hist]]) > 5) ## remove low count genes
    dataS = cr_countMat[[hist]][selectR, ]
    voomDDS[[hist]] = voom(counts = dataS, design = design, normalize.method = "cyclicloess", plot = FALSE)
    ## option 1 using voomDDS option2 using normDDS as normalized input.
    inputDDS = voomDDS[[hist]]
    ## corfit$consensus
    fit = lmFit(inputDDS, design)
    fitContrast = contrasts.fit(fit, contrast)
    fitBayes = eBayes(fitContrast, robust = TRUE)
    results[[hist]] = list()
    results_fullInfo[[hist]] = list()
  
    ## Results
    res = topTable(fit = fitBayes, adjust.method = 'fdr', coef = 1, number = nrow(inputDDS), sort = 'P') %>% data.table
    # res = data.table(PeakRegion = rownames(res), res)
    res[, Significance := ifelse((adj.P.Val <= 0.1 & sign(logFC) == 1 & abs(logFC) >= 1), 'Up',
                            ifelse((adj.P.Val <= 0.1 & sign(logFC) == -1 & abs(logFC) >= 1), 'Down', 'notDE'))]
    colnames(cr_normMat[[hist]]) = paste0(k4_proplabel$xnumber, "_", k4_proplabel$stratify_cd8_all)

    results[[hist]] = left_join(res, cr_normMat[[hist]] %>% data.frame %>% dplyr::mutate(ID = rownames(cr_normMat[[hist]])), by = "ID") %>% data.frame
    results[[hist]]$Average_LowExpander = rowMeans(results[[hist]][, which(stringr::str_detect(colnames(results[[hist]]), "Low_Expander"))])
    results[[hist]]$Average_HighExpander = rowMeans(results[[hist]][, which(stringr::str_detect(colnames(results[[hist]]), "High_Expander"))])
    results[[hist]]$Average_LowExpander_Group = 0
    results[[hist]]$Average_HighExpander_Group = 0
    results[[hist]]$Average_LowExpander_Group[results[[hist]]$Average_LowExpander > thres_list[[hist]]] = 1
    results[[hist]]$Average_HighExpander_Group[results[[hist]]$Average_HighExpander > thres_list[[hist]]] = 1
    
    write.table(results[[hist]], file = paste0(outPath, '/DE_', hist, '_TCMpatient_TSSregions_Low_High_Expander_adj0.1_logFC2.csv'), quote = FALSE, row.names = FALSE, sep = ",")
    
}


In [ ]:
## RNA-seq Differential detection
## propmatch_cd8

responseList = c("Low_Expander", "High_Expander")
hdList <- patient_id_list
target = rna_proplabel

## Experimental design
treat <- factor(target$stratify_cd8_all)
design <- model.matrix(~0 + treat)
colnames(design) <- levels(treat)
contrast <- makeContrasts(High_Low = High_Expander - Low_Expander, levels = design)


In [103]:
outpath = "CART_CUTRUN_Project/results/CUTANDRUN/analysis/PatientData/"
## Filter and delete low expressed genes
selectR = which(rowSums(rna_countMat) > 5) ## remove low count genes
dataS = rna_countMat[selectR, ]
voomDDS = voom(counts = dataS, design = design, normalize.method = "cyclicloess", plot = FALSE)

## option 1 using voomDDS option2 using normDDS as normalized input.
inputDDS = voomDDS
## corfit$consensus
fit = lmFit(inputDDS, design)
fitContrast = contrasts.fit(fit, contrast)
fitBayes = eBayes(fitContrast, robust = TRUE)

## Results
res = topTable(fit = fitBayes, adjust.method = 'fdr', coef = 1, number = nrow(inputDDS), sort = 'P')
res = data.table(ID = rownames(res), res)
res[, Significance := ifelse((adj.P.Val <= 0.1 & sign(logFC) == 1 & abs(logFC) >= 1), 'Up',
                        ifelse((adj.P.Val <= 0.1 & sign(logFC) == -1 & abs(logFC) >= 1), 'Down', 'notDE'))]
colnames(rna_normMat) = paste0(rna_proplabel$xnumber, "_", rna_proplabel$stratify_cd8_all)

rna_results = left_join(res, rna_normMat %>% data.frame %>% dplyr::mutate(ID = rownames(rna_normMat)), by = "ID") %>% data.frame
rna_results$Average_LowExpander = rowMeans(rna_results[, which(stringr::str_detect(colnames(rna_results), "Low_Expander"))])
rna_results$Average_HighExpander = rowMeans(rna_results[, which(stringr::str_detect(colnames(rna_results), "High_Expander"))])
rna_results$Average_LowExpander_Group = 0
rna_results$Average_HighExpander_Group = 0
rna_results$Average_LowExpander_Group[rna_results$Average_LowExpander > 2] = 1
rna_results$Average_HighExpander_Group[rna_results$Average_HighExpander > 2] = 1

write.table(rna_results, file = paste0(outPath, '/DE_RNAseq_TCMpatient_TSSregions_Low_High_Expander_adj0.1_logFC2.csv'), quote = FALSE, row.names = FALSE, sep = ",")




In [ ]:
RNA_CR_Merged = inner_join(inner_join(tmp1, tmp2, by = "ID"), tmp3, by = "ID")
RNA_CR_Merged[is.na(RNA_CR_Merged)] = 0
write.table(RNA_CR_Merged, file = paste0(outPath, '/DE_RNAseq_TCMpatient_TSSregions_Low_High_Expander_RNA_K4_K27_Group.csv'), quote = FALSE, row.names = FALSE, sep = ",")
